>Ouvrir le notebook dans Colab en modifiant le début de son adresse dans le navigateur :<br>
il faut remplacer **github.com** par **githubtocolab.com**.<br>
Une fois vos réponses apportées, le notebook devra être sauvegardé dans GitHub, dans le repository du TP :<br>
*Fichier > Enregistrer une copie dans Github*<br>
*Info-TSI-Vieljeux/tpx-votre_nom*<br>

---

# Tableau de pixels et images

## Importer une image

PIL (python imaging library) est l'une des librairies Python permettant de manipuler des fichiers image. On va l'utiliser en association avec numpy qui est le module de choix pour jouer avec des tableaux numériques.

In [ ]:
from PIL import Image
import urllib.request # pour récupérer une image sur le web
from IPython.display import display # pour afficher dans le notebook
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (15,10)

urllib.request.urlretrieve('https://i.redd.it/quqjmqmi44q51.jpg', 'girafe') # récupération du fichier image
image_girafe = np.array(Image.open('girafe')) # l'image est convertie en un tableau numpy
girafe = Image.fromarray(image_girafe) # le tableau est converti en un objet image
display(girafe) # affichage

On a récupéré les données de l'image dans un tableau à trois dimensions numpy. 

Les deux premières dimensions correspondent aux coordonnées spatiales du pixel et l'indexation d'un tableau numpy autorise l'utilisation de virgules pour imiter les coordonnées mathématiques (mais les x et les y sont inversés car il s'agit de matrice et on commence toujours par indexer les lignes avant les colonnes). Le pixel ayant la coordonnée cartésienne $(x,y)$ avec une origine $(0,0)$ en haut à gauche va donc correspondre à l'élément `image[y,x]` (on peut aussi, comme avec les listes python, obtenir l'élément via `image[y][x]`).

Et à chacun de ses pixels correspond un tableau de 3 entiers compris entre 0 et 255 codant la couleur du pixel (codage rgb, un nombre pour l'intensité du rouge, un pour l'intensité du vert et le dernier pour le bleu). C'est la 3<sup>e</sup> dimension du tableau.

In [ ]:
print(image_girafe.shape)

In [ ]:
hauteur, largeur, _ = image_girafe.shape
print(f'largeur : {largeur} pixels, hauteur : {hauteur} pixels')

In [ ]:
print(image_girafe[20,700])
print(image_girafe[20][700])

## Codage RGB

RGB pour Red Green Blue (RVB en français) est un système de codage informatique des couleurs. Il repose sur la synthèse additive et suit donc le même principe que le codage des couleurs dans notre cerveau à partir des signaux envoyés par trois cellules spécialisées tapissant nos rétines, les cones, chacune ayant un spectre d'absorption centré sur les longueurs d'onde correspondantes à l'une des trois couleurs, rouge, vert ou bleu.

In [ ]:
longueur = 400
synthese = np.zeros([longueur, longueur, 3], dtype=np.uint8) 
# création d'un tableau de dimension 3 (2 dimensions spatiales + la couleur) dont les entrées sont des entiers non signés codés sur 8 bits
taille = longueur//2
x1,y1 = longueur//8,5*longueur//16
x2,y2 = longueur//4,longueur//8
x3,y3 = 3*longueur//8,3*longueur//8
synthese[y1:y1+taille,x1:x1+taille] = [255, 0, 0]
synthese[y2:y2+taille,x2:x2+taille] = [0, 255, 0]
synthese[y3:y3+taille,x3:x3+taille] = [0, 0, 255]
affichage = Image.fromarray(synthese)
display(affichage)

En exécutant la cellule précédente, on voit qu'il manque à l'image les zones de superposition.

Modifier l'image `synthese` pour reproduire celle de l'énoncé.

In [ ]:
# aide : la ligne suivante ajoute la zone cyan :
synthese[y3:y2+taille,x1+taille:x2+taille] = [0, 255, 255]
# VOTRE CODE
affichage = Image.fromarray(synthese)
display(affichage)

In [ ]:
# zone de tests, ne pas modifier

Faisons notre propre expérience physique de synthèse additive en codant des zones où les pixels alternent 2 couleurs primaires :

In [ ]:
largeur = 900
hauteur = 300
synth_phy = np.zeros([hauteur, largeur, 3], dtype=np.uint8)
for i in range(hauteur) :
    for j in range(largeur//3) :
        if (i+j)%2 :
            synth_phy[i,j] = [255,0,0]
        else : 
            synth_phy[i,j] = [0,255,0]
for i in range(hauteur) :
    for j in range(largeur//3,2*largeur//3) :
        if (i+j)%2 :
            synth_phy[i,j] = [0,0,255]
        else : 
            synth_phy[i,j] = [0,255,0]
for i in range(hauteur) :
    for j in range(2*largeur//3,largeur) :
        if (i+j)%2 :
            synth_phy[i,j] = [0,0,255]
        else : 
            synth_phy[i,j] = [255,0,0]
synth_phy = Image.fromarray(synth_phy)
display(synth_phy)

Récupérons les composantes bleues, vertes et rouges de la photo de girafe :

In [ ]:
image_R = image_girafe.copy()
image_R[:,:,(1,2)] = 0
# équivalent à :
# for i in range(hauteur) :
#     for j in range(largeur) :
#         image_R[i][j][1] = 0
#         image_R[i][j][2] = 0
image_G = image_girafe.copy()
image_G[:,:,(0,2)] = 0
image_B = image_girafe.copy()
image_B[:,:,(0,1)] = 0

rvb = np.concatenate((image_R, image_G, image_B), axis=1)
rvb = Image.fromarray(rvb)
display(rvb)

La superposition des trois filtres reproduit les couleurs d'origine.

In [ ]:
image_rec = image_B.copy()
image_rec[:,60:500] += image_G[:,60:500]
image_rec[75:650,:] += image_R[75:650,:]
image_rec = Image.fromarray(image_rec)
display(image_rec)

Dans le codage RGB utilisé aujourd'hui, l'intensité de chacune des 3 couleurs primaires est codée sur un octet (8 bits), ce qui permet une profondeur de 24 bits pour différentier les couleurs.<br>
Combien de couleurs sont alors représentables par ce système ?

In [ ]:
# Modifiez l'entier affecté à la variable `nb_couleurs` pour qu'il corresponde à votre réponse :
nb_couleurs = 25

In [ ]:
# zone de tests, ne pas modifier

Fabriquons une image contenant toutes ces couleurs.

L'idée est de fabriquer d'abord une image $256\times 256$ contenant toutes les nuances possibles de vert et rouge, de l'agrandir d'un facteur 16 de manière à qu'une combinaison rouge/vert unique corresponde à un gros pixel de $16\times16$. Et on additionne à chacun de ces gros pixels une image $16\times16$ bleu contenant les 256 teintes de bleu disposées en spirale.

![](http://cordier-phychi.toile-libre.org/Info/github/rougevertetspir.png)

L'image rouge/verte est assez simple à coder (l'exécution met un peu de temps) :

In [ ]:
L1 = 4096
rougevert = np.zeros([L1, L1, 3], dtype=np.uint8)
for r in range(256*16) :
    for g in range(256*16) :
        rougevert[r,g]=[r//16,g//16,0]
plt.imshow(rougevert)

Fabriquer la spirale bleue est plus dur...<br> 
L'image doit faire $16\times16$ et contenir toutes les nuances de bleu. On commence en haut à gauche (x=0 et y=0) par du noir (0,0,0), et on progresse dans le sens des aiguilles d'une montre en ajoutant 1 à l'intensité du bleu à chaque pixel successif de la spirale pour finir au centre (en position x=7, y=8 pour être précis) par un pixel 100% bleu (0,0,255).

Construisez la spirale bleue (du moins la matrice qui sera représentée par la spirale bleue).

In [ ]:
# vous appellerez votre image "bleu"
L2 = 16
bleu = np.zeros([L2, L2, 3], dtype=np.uint8)
# VOTRE CODE
plt.imshow(bleu)
# cette commande devra afficher une image similaire à celle de l'énoncé !

In [ ]:
# zone de tests, ne pas modifier

In [ ]:
longueur = 4096
rougevertbleu = np.zeros([longueur, longueur, 3], dtype=np.uint8)
for i in range(0,4096,16) :
    for j in range(0,4096,16) :
        rougevertbleu[i:i+16,j:j+16] = rougevert[i:i+16,j:j+16]+bleu
rougevertbleu = Image.fromarray(rougevertbleu)
display(rougevertbleu)

Les premières consoles de jeu avaient des graphismes de 6 bits (de profondeur). Plutôt que 256 possibilités pour chaque sous-pixel, on en était réduit à seulement 4 choix (2 bits).

![](https://d3isma7snj3lcx.cloudfront.net/optim/images/gallery/26/26635/super-mario-bros-nes-155dee7e__930_250__86-619-1680-1050.jpg)

Définissez une fonction permettant de convertir l'image de la giraffe en 6 bits.

In [ ]:
def sixbit(image) :
    """
    prend en argument un tableau numpy à 3 dimensions () obtenu comme plus haut via np.array(Image.open('nom'))
    et renvoie un nouveau tableau de mêmes dimensions correspondant à une conversion en 6 bits de l'image (chacune des 3 couleurs doit maintant n'avoir que 4 valeurs d'intensité possibles uniformément réparties).
    """
    # VOTRE CODE

In [ ]:
giraffe = Image.fromarray(sixbit(image_girafe))
display(giraffe)

In [ ]:
# zone de tests, ne pas modifier

On peut très facilement inverser les couleurs de l'image. Une ligne suffit :

In [ ]:
image_inv = 255-image_girafe
image_inv = Image.fromarray(image_inv)
display(image_inv)

Construire une fonction `NB` qui retourne une version "niveau de gris" de l'image donnée en argument.

Principe de la manœuvre : $(r,g,b)\rightarrow (\frac{r+g+b}{3},\frac{r+g+b}{3},\frac{r+g+b}{3})$

In [ ]:
def NB(image) :
    """
    prend en argument un tableau numpy à 3 dimensions () obtenu comme plus haut via np.array(Image.open('nom'))
    et renvoie un nouveau tableau correspondant à une conversion en niveau de gris de l'image.
    """
    # VOTRE CODE

In [ ]:
girafe_NB = Image.fromarray(NB(image_girafe))
display(girafe_NB)

In [ ]:
# zone de tests, ne pas modifier

## Transformation d'une image

Construisez une fonction `recadrage` qui prend en argument l'image à recadrer, les coordonnées du coin supérieur gauche du nouveau cadre (sous la forme d'un tuple (x,y)), la largeur et la hauteur du nouveau cadre.<br>
Faites en sorte que `recadrage(image_girafe,(30,50),500,600)` recadre la tête de la girafe comme ci-dessous.
<img src="http://cordier-phychi.toile-libre.org/Info/github/recadrage.png" width="800"/>

In [ ]:
def recadrage(image,xy_coin,l_cadre,h_cadre) :
    """
    prend en argument un tableau numpy à 3 dimensions (hauteur,largeur,3) obtenu comme plus haut via np.array(Image.open('nom'))
    et renvoie un nouveau tableau à 3 dimensions (h_cadre,l_cadre,3).
    L'argument xy_coin est un tuple (x,y) où x et y sont des entiers correspondants aux coordonnées du coin supérieur gauche du nouveau cadre.
    l_cadre et h_cadre étant des nombres de pixels, ils doivent être entiers.
    """
    # VOTRE CODE

In [ ]:
affichage = Image.fromarray(recadrage(image_girafe,(30,50),500,600))
display(affichage)

In [ ]:
# zone de tests, ne pas modifier

Construisez ensuite une fonction `rotation` qui tourne l'image de 90° vers la gauche en modifiant la disposition des pixels.

In [ ]:
def rotation(image) :
    """
    prend en argument un tableau numpy à 3 dimensions (hauteur,largeur,3) obtenu comme plus haut via np.array(Image.open('nom'))
    et renvoie un nouveau tableau correspondant à l'image tournée de 90° vers la gauche.
    """
    # VOTRE CODE

In [ ]:
affichage = Image.fromarray(rotation(image_girafe))
display(affichage)

In [ ]:
# zone de tests, ne pas modifier

On peut aussi s'amuser avec les symétries :

In [ ]:
image_sym = np.copy(image)
for i in range(min(hauteur,largeur)) :
    for j in range(min(hauteur,largeur)) :
        image_sym[i][j]=image_sym[j][i]
affichage = Image.fromarray(image_sym)
display(affichage)

## Traitement d'image (filtrage)

On va maintenant passer à des transormations plus évoluées : 
- flou
- amélioration de la netteté
- détection des contours

Elles reposent sur des convolutions dont la recette est la suivante :
- une petite matrice, la matrice de convolution, appelée **noyau**, est choisie,
- on balaye l'image à traiter avec un cadre ayant la taille de la matrice,
- à l'intérieur du cadre, on multiplie chacune des valeurs d'intensité des pixels par le coefficient correspondant de la matrice,
- on somme tous ces produits et on attribue la valeur au pixel au centre du cadre.

Suivant le noyau utilisé, on va modifier l'image de différentes façons.

### Floutage

L'idée va être de moyenner la valeur des pixels à l'intérieur du bloc grâce à la matrice F suivante :
$$F=\frac{1}{9}\begin{pmatrix}1&1&1\\1&1&1\\1&1&1\end{pmatrix}$$
Cela revient à passer l'image dans un filtre passe-bas (= un moyenneur).<br>
En effet, la valeur de l'intensité d'un pixel sera maintenant une moyenne entre tous ses voisins.

In [ ]:
def conv_np(A,B) :
    h = len(A)
    l = len(A[0])
    C = np.zeros((h-2, l-2),dtype=int)
    for i in range(3) :
        for j in range(3) :
            C += B[i,j]*A[i:h-2+i,j:l-2+j]
    return C

In [ ]:
A = np.array([[2,2,2,2,2,2],
              [2,1,1,1,2,2],
              [2,1,3,4,2,2],
              [2,1,2,1,2,2],
              [2,2,3,2,2,2]])

B = np.array([[1,0,1],
              [0,2,0],
              [1,0,1]])

C = conv_np(A,B)
print(C)

Que valent les éléments $c_{ij}$ de la matrice $C$ donnée par `conv_np(A,B)` ?
- a : $\sum_{k=1}^{3}a_{ik}b_{kj}$
- b : $\sum_{i=1}^{3}\sum_{j=1}^{3}a_{(5-i)(5-j)}b_{ij}$ 
- c : $\sum_{i=1}^{3}\sum_{j=1}^{3}a_{(i+1)(j+1)}b_{ij}$ 

In [ ]:
# Affectez à la variable rep le caractère correspondant à votre réponse parmi 'a', 'b' ou 'c' :
rep = 'd'

In [ ]:
# zone de tests, ne pas modifier

La fonction suivante réalise la même tâche. Elle est moins rapide, mais permet des matrices $B$ plus grandes que $3\times 3$ grâce à l'introduction du paramètre `marge`.

In [ ]:
def conv(A,B,marge) :
    A = A.astype(np.int16)
    hauteur, largeur = A.shape
    C = A.copy()
    for i in range(marge,hauteur-marge) :
        for j in range(marge,largeur-marge) :
            C[i,j] = (A[i-marge:i+marge+1,j-marge:j+marge+1]*B).sum()
    return C

On va ainsi pouvoir contrôler l'intensité du floutage en liant la taille de la matrice $B$ au paramètre `force`.

In [ ]:
def flou(image,force) :
    """
    flou(image,intensite_flou)->image_floue
    image doit être un tableau dimension d'entier non signés codés sur 8 bits
    intensité_flou est un entier >= 1
    image_floue est du même type qu'image
    """
    taille = 2*force+1 # taille de la matrice F
    F = np.ones((taille,taille))*1/taille**2  # matrice pour la convolution
    marge = (taille-1)//2
    image_floue = image.copy()
    hauteur,largeur = image_floue.shape
    image_floue = conv(image,F,marge)
    image_floue = image_floue.astype(np.uint8)
    return image_floue

In [ ]:
urllib.request.urlretrieve('https://fichier0.cirkwi.com/image/photo/poi/800x500/545297/fr/0.jpg', 'LaR')
image_LR = np.array(Image.open('LaR'))
hauteur,largeur,_ = image_LR.shape
LaR = np.zeros([hauteur, largeur]) 
# on associe maintenant à chaque pixel un seul chiffre : l'intensité de gris (entre 0 et 255)
LaR = NB(image_LR)[:,:,0] # il suffit de récupérer une des 3 couleurs de la conversion en niveau de gris de l'image
affichage = Image.fromarray(LaR)
display(affichage)

In [ ]:
LaR_floues = (LaR,)  # un singulet nécessite cette petite virgule pour être reconnu comme tel
for i in range(1,5) :
    LaR_floues += (flou(LaR,i),)
comparaison = np.concatenate(LaR_floues, axis=1)
affichage = Image.fromarray(comparaison)
display(affichage)

Les matrices F utilisées dans les 4 images floutées :
$\frac{1}{9}\begin{pmatrix}1&1&1\\1&1&1\\1&1&1\end{pmatrix}$,$\frac{1}{16}\begin{pmatrix}1&1&1&1\\1&1&1&1\\1&1&1&1\\1&1&1&1\end{pmatrix}$,$\frac{1}{25}\begin{pmatrix}1&1&1&1&1\\1&1&1&1&1\\1&1&1&1&1\\1&1&1&1&1\\1&1&1&1&1\end{pmatrix}$,$\frac{1}{36}\begin{pmatrix}1&1&1&1&1&1\\1&1&1&1&1&1\\1&1&1&1&1&1\\1&1&1&1&1&1\\1&1&1&1&1&1\\1&1&1&1&1&1\end{pmatrix}$.<br>
Plus la matrices est grande, plus on moyenne de points voisins, plus le flou est important...

### Amélioration de la netteté

On ne veut maintenant plus moyenner, mais au contraire accentuer les différences (cela revient à passer l'image dans un filtre passe-haut).<br>
Pour cela, on choisit un noyau $N$ qui récompense les variations entre pixels voisins et est sans effet dans les zones de mêmes teintes :
$$N=\begin{pmatrix}0&-1&0\\-1&5&-1\\0&-1&0\end{pmatrix}$$

Sur le modèle de la fonction `flou`, mais en plus simple, car pas besoin ici de s'embêter avec une marge variable (`conv_np` est donc utilisable), complétez la définition de la fonction `net` qui renvoie le résultat d'une image convoluée par $N$.

In [ ]:
def net(image) :
    """
    net(image)->image_nette
    """
    image = image.astype(np.int32)
    # VOTRE CODE
    # on fixe les valeurs qui ont dépassé 255 à 255 et celles sous 0 à 0.
    image_nette[image_nette<0] = 0
    image_nette[image_nette>255] = 255
    image_nette = image_nette.astype(np.uint8) 
    return image_nette

In [ ]:
LaR_nette = net(LaR_floues[1])
comparaison = np.concatenate((LaR_floues[1][1:-1,1:-1],LaR_nette), axis=1)
affichage = Image.fromarray(comparaison)
display(affichage)

In [ ]:
# zone de tests, ne pas modifier

### Détection de contour (filtre de Sobel)

On va agir en deux temps, grâce à deux noyaux.<br>
L'un, $S_x$, va donner des valeurs d'autant plus loin de $0$ qu'il y a un fort gradient horizontal d'intensité dans le bloc $3\times3$ de l'image inspectée.<br>
Et l'autre, $S_y$, va mettre en valeur les gradients verticaux.<br>
$S_x = \begin{pmatrix}-1&0&1\\-2&0&2\\-1&0&1\end{pmatrix}$ et $S_y = \begin{pmatrix}1&2&1\\0&0&0\\-1&-2&-1\end{pmatrix}$<br>
$S_x$ fait la différence entre les voisins de droite et ceux de gauche quand $S_y$ fait la différence entre les voisins du dessus et ceux de dessous.

In [ ]:
def grad_x(image) :
    image = image.astype(np.int32)
    Sx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    image_gradx = conv_np(image,Sx)
    # les gradients peuvent très bien être négatifs. On translate alors toutes les valeurs pour que la plus basse soit zéro.
    image_gradx = image_gradx - np.min(image_gradx)
    # on normalise ensuite en faisant en sorte que la plus haute valeur vaille 255
    image_gradx = image_gradx/np.max(image_gradx)*255
    image_gradx = image_gradx.astype(np.uint8)
    return image_gradx

Écrivez la fonction `grad_y` sur le même modèle :

In [ ]:
def grad_y(image) :
    # VOTRE CODE

In [ ]:
Gx = grad_x(LaR)
Gy = grad_y(LaR)
comparaison = np.concatenate((Gx,Gy), axis=0)
affichage = Image.fromarray(comparaison)
display(affichage)

In [ ]:
# zone de tests, ne pas modifier

Le gradient global $G$ s'obtient en "pythagorisant" `Gx` et `Gy` : $G=\sqrt{G_x^2+G_y^2}$

Remarque : cela revient finalement à appliquer un filtre passe-haut à l'image.

In [ ]:
def grad(image) :
    Gx = grad_x(image).astype(np.int32)
    Gy = grad_y(image).astype(np.int32)
    image_grad = np.sqrt(Gx**2+Gy**2)
    image_grad = image_grad/np.max(image_grad)*255
    image_grad = image_grad.astype(np.uint8)
    return image_grad

In [ ]:
affichage = Image.fromarray(grad(LaR))
display(affichage)

L'effet de relief est rendu par l'information sur la direction du gradient, information inutile si le contour est tout ce qui nous intéresse (que l'on passe d'une forte intensité à une faible ou l'inverse détecte un contour dans les deux cas).<br>
On va donc reprendre les définitions en prenant les valeurs absolues des gradients.

In [ ]:
def grad_abs_x(image) :
    image = image.astype(np.int32)
    Sx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    image_gradx = np.abs(conv_np(image,Sx))
    image_gradx = image_gradx/np.max(image_gradx)*255
    image_gradx = image_gradx.astype(np.uint8)
    return image_gradx
def grad_abs_y(image) :
    image = image.astype(np.int32)
    Sy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    image_grady = np.abs(conv_np(image,Sy))
    image_grady = image_grady/np.max(image_grady)*255
    image_grady = image_grady.astype(np.uint8)
    return image_grady
def contour(image) :
    Gx = gradabs_x(image).astype(np.int32)
    Gy = gradabs_y(image).astype(np.int32)
    image_cont = np.sqrt(Gx**2+Gy**2)
    image_cont = image_cont/np.max(image_cont)*255
    image_cont = image_cont.astype(np.uint8)
    return image_cont

In [ ]:
affichage = Image.fromarray(contour(LaR))
display(affichage)

Parmi les images suivantes numérotées de 1 à 4, laquelle est produite par :
- A : `grad_x(echiquier)`
- B : `grad_abs_y(echiquier)`
- C : `contour(echiquier)`
- D : `grad(echiquier)`
où `echiquier` est l'image suivante :

![](http://cordier-phychi.toile-libre.org/Info/github/echiquier.png)

<img src="http://cordier-phychi.toile-libre.org/Info/github/echiquiz.png" alt="" width="640"/>

In [ ]:
# Affectez chacune des variables `A`, `B`, `C`, `D` à l'entier `1`,`2`,`3` ou `4` correspondant à la bonne image.
A = 0
B = 0
C = 0
D = 0

In [ ]:
# zone de tests, ne pas modifier

Quand on joue avec des images, les erreurs de code donne parfois des résultats étonnants. N'hésitez pas à enregistrer/copier vos bizarreries, s'il y en a. Je récompenserai la plus belle/tordue.<br>
Ci-dessous, un échec faisant tomber la pluie sur La Rochelle...
![](http://cordier-phychi.toile-libre.org/Info/github/pluieLR.png)

In [ ]:
image_LR = np.array(Image.open('Rochelle.jpeg'))
hauteur,largeur,_ = image_LR.shape
LaR = np.zeros([hauteur, largeur]) 
# on associe maintenant à chaque pixel un seul chiffre : l'intensité de gris (entre 0 et 255)
LaR = NB(image_LR)[:,:,0] # il suffit de récupérer une des 3 couleurs de la conversion en niveau de gris de l'image
affichage = Image.fromarray(LaR)
display(affichage)

In [ ]:
LaR_floue = flou(LaR,2)
affichage = Image.fromarray(LaR_floue)
display(affichage)

In [ ]:
LaR_nette = net(LaR_floue)
affichage = Image.fromarray(LaR_nette)
display(affichage)

In [ ]:
image_LR = np.array(Image.open('LaRtrame.png'))
hauteur,largeur,_ = image_LR.shape
LaR = np.zeros([hauteur, largeur]) 
# on associe maintenant à chaque pixel un seul chiffre : l'intensité de gris (entre 0 et 255)
LaR = NB(image_LR)[:,:,0] # il suffit de récupérer une des 3 couleurs de la conversion en niveau de gris de l'image
affichage = Image.fromarray(LaR)
display(affichage)

In [ ]:
from scipy import fftpack
import copy
import matplotlib.cm
im_fft = fftpack.fft2(LaR)
def plot_spectrum(im_fft):
    from matplotlib.colors import LogNorm
    # A logarithmic colormap
    plt.imshow(np.abs(im_fft), norm=LogNorm(vmin=1, vmax=1e8),cmap=my_cmap)
    plt.colorbar()

plt.figure(dpi=300)
plot_spectrum(im_fft)
plt.title('Fourier transform')

In [ ]:
copie_fft = im_fft.copy()
H, L = copie_fft.shape
l = 5
for i in range(1,20) :
    if i != 10 :
        yop = np.tile(copie_fft[:, L//20*i-2*l], (2*l, 1)).T
        print(i,copie_fft[:, L//20*i-l:L//20*i+l].shape,yop.shape,L//20*i-l,L//20*i+l)
        copie_fft[:, L//20*i-l:L//20*i+l] = yop
#copie_fft[:,:l] = 0

In [ ]:
np.tile(copie_fft[:, L//20*i-2*l], (4, 1)).T.shape

In [ ]:
np.array(list(copie_fft[:, L//20*i-2*l])*2*l)

In [ ]:
copie_fft = im_fft.copy()
copie_fft[:, L//20*5-l:L//20*5+l].shape

In [ ]:
plt.figure(dpi=300)
plot_spectrum(copie_fft)
plt.title('Filtered Spectrum')

In [ ]:
im_new = fftpack.ifft2(copie_fft).real
plt.figure(dpi=400)
plt.imshow(im_new, plt.cm.gray)
plt.title('Reconstructed Image')

In [ ]:
im_new